this file is generated by freecodecamp algo tranding session
https://github.com/nickmccullum/algorithmic-trading-python/blob/master

In [ ]:
pip install pandas
pip install numpy
pip install xlsxwriter

In [1]:
import numpy as np
import pandas as pd
import math
import requests
import xlsxwriter

read list of 500 stocks

In [40]:
stocks=pd.read_csv('sp_500_stocks.csv')

aquire API token

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

making first call

In [9]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
api_url

'https://sandbox.iexapis.com/stable/stock/AAPL/quote/?token=Tpk_059b97af715d417d9f49f50b51b1c448'

In [ ]:
data=requests.get(api_url).json()
data

In [13]:
price=data['latestPrice']
marketcap=data['marketCap']/1000000000000


2.312874104675

In [14]:
my_columns=['ticker','stockprice','marketcap','numberofsharetobuy']
final_dataframe=pd.DataFrame(columns=my_columns)

In [17]:
final_dataframe.append(
    pd.Series(
        [symbol,price,marketcap,'N/A'] , index=my_columns
    ), ignore_index=True
)

,ticker,stockprice,marketcap,numberofsharetobuy
0,AAPL,145.11,2.312874,N/A


In [22]:
final_dataframe=pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:2]:
    api_url=f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series(
        [stock,data['latestPrice'],data['marketCap'],'N/A'] , index=my_columns
    ), ignore_index=True
    )

In [23]:
final_dataframe

,ticker,stockprice,marketcap,numberofsharetobuy
0,A,129.36,38686628710,N/A
1,AAL,12.62,8302898463,N/A


using batch api mode

In [24]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [41]:
symbolgroups=list(chunks(stocks['Ticker'],100))
symbolstrings=[]
for i in range(0,len(symbolgroups)):
    symbolstrings.append(','.join(symbolgroups[i]))

final_dataframe=pd.DataFrame(columns=my_columns)
for symbolstrings in symbolstrings:
    batch_api_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbolstrings}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_url).json()
    for symbol in symbolstrings.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [symbol,data[symbol]['quote']['latestPrice'],data[symbol]['quote']['marketCap'],'N/A'] , index=my_columns
            ), ignore_index=True
        )
final_dataframe

,ticker,stockprice,marketcap,numberofsharetobuy
0,A,129.83,39028926545,N/A
1,AAL,12.24,7953559784,N/A
2,AAP,166.18,10029751955,N/A
3,AAPL,144.27,2269416227813,N/A
4,ABBV,139.46,251475783166,N/A
...,...,...,...,...
496,YUM,113.64,31220509331,N/A
497,ZBH,110.46,23474741878,N/A
498,ZBRA,279.11,14280506814,N/A
499,ZION,51.96,7789815947,N/A


In [45]:
portfolio_size=int(input('enter portfolio size'))


In [46]:
position_size=portfolio_size/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'numberofsharetobuy']=math.floor(position_size/final_dataframe.loc[i,'stockprice'])


writing to excel

In [48]:
writer=pd.ExcelWriter('recommended_trade.xlsx',engine='xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades',index=False)

formatting excel

In [49]:
writer.save()

#yt run time 1:38